In [399]:
import numpy as np
from PIL import Image


In [400]:
def loadImage(path):
    img = Image.open(path)
    
    # 将图像转换成灰度图
    img = img.convert("L")
    # 图像的大小在size中是（宽，高）
    # 所以width取size的第一个值，height取第二个
    width = img.size[0]
    height = img.size[1]
    data = img.getdata()
    
    # 为了避免溢出，这里对数据进行一个缩放，缩小100倍
    data = np.array(data).reshape(height,width)/100
    #print(data.shape)
    # 查看原图的话，需要还原数据
    #new_im = Image.fromarray(data*100)
   # new_im.show()
    return data

In [401]:
name1 = "Camera Roll/"

In [402]:
#对100张图片进行进行切片处理，并把全部图片整合到IMG中
IMG =(np.zeros((3174,100),dtype = float))
for id in range(1,101):
    filename = name1+str(id)+".jpg"
    
    data = loadImage(filename)
    data = data[120:189,190:236]
    data = data.reshape(3174)
    IMG[:,id-1]=data
IMG

array([[0.4 , 0.48, 0.44, ..., 1.53, 1.45, 1.7 ],
       [0.51, 0.5 , 0.52, ..., 1.83, 1.75, 1.86],
       [0.46, 0.46, 0.46, ..., 1.82, 1.68, 1.9 ],
       ...,
       [2.14, 2.2 , 2.2 , ..., 0.74, 0.86, 0.79],
       [2.29, 2.24, 2.17, ..., 0.79, 0.89, 0.82],
       [2.1 , 1.98, 1.98, ..., 0.81, 0.89, 0.85]])

In [404]:
#对每张图片归一化
mean = IMG.sum(axis=1)/100
for i in range(0,100):
    IMG[:,i] -= mean
IMG

array([[-0.2428, -0.1628, -0.2028, ...,  0.8872,  0.8072,  1.0572],
       [-0.1631, -0.1731, -0.1531, ...,  1.1569,  1.0769,  1.1869],
       [-0.2256, -0.2256, -0.2256, ...,  1.1344,  0.9944,  1.2144],
       ...,
       [ 0.1871,  0.2471,  0.2471, ..., -1.2129, -1.0929, -1.1629],
       [ 0.3103,  0.2603,  0.1903, ..., -1.1897, -1.0897, -1.1597],
       [ 0.3065,  0.1865,  0.1865, ..., -0.9835, -0.9035, -0.9435]])

In [405]:
#生成自相关矩阵R
R_matrix = (np.zeros((3174,3174),dtype = float))
for i in range(0,100):
    temp = np.array(IMG[:,i]).reshape(3174,1)
    R_matrix+=np.dot(temp,temp.transpose())/100
R_matrix.shape
R_matrix


array([[ 0.21261416,  0.23819432,  0.24007232, ..., -0.24556912,
        -0.24819216, -0.2030938 ],
       [ 0.23819432,  0.27472939,  0.27618764, ..., -0.28306899,
        -0.28611507, -0.23415185],
       [ 0.24007232,  0.27618764,  0.28009664, ..., -0.28570224,
        -0.28917132, -0.2366066 ],
       ...,
       [-0.24556912, -0.28306899, -0.28570224, ...,  0.29458659,
         0.29738787,  0.24368485],
       [-0.24819216, -0.28611507, -0.28917132, ...,  0.29738787,
         0.30179691,  0.24667905],
       [-0.2030938 , -0.23415185, -0.2366066 , ...,  0.24368485,
         0.24667905,  0.20325875]])

In [406]:
value,vector = np.linalg.eig(R_matrix)# 生成特征值和特征向量

In [408]:
print(value.shape)
vector.shape
np.dot(vector.transpose(),vector)#验算一下特征向量是否正确

(3174,)


array([[ 1.00000000e+00+0.00000000e+00j,  2.94902991e-16+0.00000000e+00j,
         6.02816408e-17+0.00000000e+00j, ...,
        -1.02999206e-17-4.22838847e-18j, -1.02999206e-17+4.22838847e-18j,
        -5.55653613e-18+0.00000000e+00j],
       [ 2.94902991e-16+0.00000000e+00j,  1.00000000e+00+0.00000000e+00j,
         4.16333634e-17+0.00000000e+00j, ...,
        -8.99887803e-18-6.28837260e-18j, -8.99887803e-18+6.28837260e-18j,
         2.16840434e-18+0.00000000e+00j],
       [ 6.02816408e-17+0.00000000e+00j,  4.16333634e-17+0.00000000e+00j,
         1.00000000e+00+0.00000000e+00j, ...,
         1.34441069e-17-1.44740990e-17j,  1.34441069e-17+1.44740990e-17j,
         2.03558958e-17+0.00000000e+00j],
       ...,
       [-1.02999206e-17-4.22838847e-18j, -8.99887803e-18-6.28837260e-18j,
         1.34441069e-17-1.44740990e-17j, ...,
         2.63291602e-01-4.51632889e-01j,  1.00000000e+00+0.00000000e+00j,
         2.37877983e-01-1.44074909e-01j],
       [-1.02999206e-17+4.22838847e-18j, -8.

In [514]:
#输出前k大个特征值的下标
top_k = 20
top_k_idx=value.argsort()[-top_k:][::-1]
print(top_k_idx)
value

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


array([7.00667987e+01+0.00000000e+00j, 2.23468380e+01+0.00000000e+00j,
       7.74993387e+00+0.00000000e+00j, ...,
       1.39131548e-19+9.92081256e-20j, 1.39131548e-19-9.92081256e-20j,
       4.21703518e-19+0.00000000e+00j])

In [515]:
#将前k个特征值对应的特征向量整合到ve_matrix
ve_matrix=np.zeros((3174,top_k))
for i in range(0,top_k):
    ve_matrix[:,i] = vector[:,top_k_idx[i]]
print(ve_matrix.shape)

(3174, 20)


D:\anaconda\lib\site-packages\ipykernel_launcher.py:3: ComplexWarning: Casting complex values to real discards the imaginary part
  This is separate from the ipykernel package so we can avoid doing imports until


In [546]:
np.dot(ve_matrix.transpose(),ve_matrix)

array([[ 1.00000000e+00,  3.19189120e-16,  6.93889390e-17,
         1.04083409e-17, -5.55111512e-17,  1.16226473e-16,
         6.24500451e-17,  4.16333634e-17, -3.46944695e-18,
         6.93889390e-17, -1.73472348e-17, -3.12250226e-17,
         1.02348685e-16, -1.04083409e-17,  0.00000000e+00,
         2.77555756e-17,  6.93889390e-18,  3.64291930e-17,
        -1.95156391e-17, -4.51028104e-17],
       [ 3.19189120e-16,  1.00000000e+00,  6.93889390e-17,
        -2.35922393e-16, -3.46944695e-17, -4.16333634e-17,
        -1.38777878e-17,  4.16333634e-17,  9.02056208e-17,
         9.54097912e-17, -1.31838984e-16, -1.17961196e-16,
         3.46944695e-18,  1.38777878e-17, -6.93889390e-18,
         2.25514052e-16, -6.24500451e-17, -7.97972799e-17,
        -1.38777878e-17,  1.38777878e-17],
       [ 6.93889390e-17,  6.93889390e-17,  1.00000000e+00,
         1.94982919e-15, -2.77555756e-17, -2.08166817e-17,
        -1.33573708e-16,  3.33066907e-16,  2.91433544e-16,
        -2.77555756e-17,  2.0

In [547]:
#随便选一张图片，将其恢复
test_data = IMG[:,80]+mean

In [548]:
#求出Y，并将生成的人脸基图像修改
Y = np.dot(ve_matrix.T,test_data)
print(Y)
Y[5]+=2

[ -5.58869071 -13.8680006    8.32227941  -3.10849202   6.84013081
 -10.95396279  -3.03602037  13.42915576  -6.60361126   7.15777543
   8.91014433  12.26477306  -5.82437185   6.79044291   7.89325199
   8.54668302   2.85643397  -8.07823698  -9.20752138  -0.28684093]


In [549]:
#生成最后的结果
final = np.dot(ve_matrix,Y)
print(final.shape)
print(final)

(3174,)
[-1.32292946 -1.23518186 -1.18547295 ...  1.35148311  1.45605902
  1.32423651]


In [550]:
print(final)
final = np.array(final,dtype='float')
final.shape

[-1.32292946 -1.23518186 -1.18547295 ...  1.35148311  1.45605902
  1.32423651]


(3174,)

In [551]:
print(test_data)


[0.44 0.41 0.5  ... 2.2  2.22 2.02]


In [1]:
#输出结果图像
final = final.reshape(69,46)
new_im = Image.fromarray(final*100)
new_im.show()

NameError: name 'final' is not defined

In [545]:
#输出原图像
test_data= test_data.reshape(69,46)
new_test = Image.fromarray(test_data*100)
new_test.show()